In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

|General <br />  Notation  | Description| Python (if any) |
|:-------------|:------------------------------------------------------------||
| $r(i,j)$     | scalar; = 1  if user j rated movie i  = 0  otherwise             ||
| $y(i,j)$     | scalar; = rating given by user j on movie  i    (if r(i,j) = 1 is defined) ||
|$\mathbf{w}^{(j)}$ | vector; parameters for user j ||
|$b^{(j)}$     |  scalar; parameter for user j ||
| $\mathbf{x}^{(i)}$ |   vector; feature ratings for movie i        ||     
| $n_u$        | number of users |num_users|
| $n_m$        | number of movies | num_movies |
| $n$          | number of features | num_features                    |
| $\mathbf{X}$ |  matrix of vectors $\mathbf{x}^{(i)}$         | X |
| $\mathbf{W}$ |  matrix of vectors $\mathbf{w}^{(j)}$         | W |
| $\mathbf{b}$ |  vector of bias parameters $b^{(j)}$ | b |
| $\mathbf{R}$ | matrix of elements $r(i,j)$                    | R |

Generate two vectors: 
1. For each user (column) - parameter vector 
2. For each movie (row) - feature vector 

Dot product of these two vectors (+bias term) produce an estimate of the rating the user might give to that movie

# 

In [4]:
#Loading the dataset

X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("num_features: ", num_features)
print("num_movies: ",   num_movies)
print("num_users: ",    num_users)

print("Y shape: ", Y.shape, "R shape: ", R.shape) # movies x users (same shape but R is binary)
print("X shape: ", X.shape) #movies x features
print("W shape: ", W.shape) #users x features


num_features:  10
num_movies:  4778
num_users:  443
Y shape:  (4778, 443) R shape:  (4778, 443)
X shape:  (4778, 10)
W shape:  (443, 10)


In [5]:
#Average rating

tsmean = np.mean(Y[0, R[0,:].astype(bool)])
print(f"Average rating for movie 1: {tsmean:0.3f} / 5")

Average rating for movie 1: 3.400 / 5


### Collaborative filtering learning algorithm

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\underbrace{
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}_{regularization}
\tag{1}$$

In [8]:
#Computing the cost function

def cofi_cost_func(X,W, b, Y, R, lambda_):
    nm, nu = Y.shape
    J = 0
    for j in range(nu):
        w = W[j,:] #All w parameters for user j
        b_j = b[0,j] #b paramater for user j
        
        for i in range(nm):
            x = X[i,:] #All features for movie i
            r = R[i,j] #Did user j rated movie i
            y = Y[i,j] #Rating for movie i from a user j
            
            
            J+= np.square(r * (np.dot(w,x) + b_j - y)) 
    J = J/2
    J+= (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))

    return J

In [9]:
#Reducing dataset size for faster testing
num_users_r = 4
num_movies_r = 5
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function without regularization
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost without regularization: {J:0.2f}")

#Evaluate cost function with regularization
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost without regularization: {J:0.2f}")


Cost without regularization: 13.67
Cost without regularization: 28.09


In [10]:
#Vectorized implementation

def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [ ]:
#Testing Vectorized implementation

# Evaluate cost function
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluate cost function with regularization 
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

In [ ]:
#Learning movie recommendations

movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)

In [ ]:
# We have selected a few movies we liked / did not like and the ratings we gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)

my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]


print('\nNew user ratings: \n')

for i in range(len(my_ratings)):
    if my_ratings[i]>0:
        print(f"Rated {my_ratings[i]} for {movieList_df.loc[i,'title']}")

In [ ]:
# Reload ratings and add new ratings
Y, R = load_ratings_small()
Y    = np.c_[my_ratings, Y]
R    = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

In [ ]:
#Preparing the model

nuM_movies, num_users = Y.shape
num_features = 100
tf.random.set_seed(1234) #for consistent results

#Setting parameters
W = tf.Variable(tf.random.normal((num_users, num_features), dtype=tf.float64), name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features), dtype=tf.float64), name='X')
b = tf.Variable(tf.random.normal((1, num_users), dtype=tf.float64), name='b')

#Instantiate an optimizer

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
#Training

iterations = 200
lambda_ = 1

for iter in range(iterations):
    with tf.GradientTape() as tape:
        #Compute the cost
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)
    
    #Automatically get the gradients of the trainable variables 
    grads = tape.gradient(cost_value, [X, W, b])
    
    #Run one step of Gradient Descent by updating the valuo of the variables to minimize the loss
    optimizer.apply_gradients(zip(grads, [X, W, b]))
    
    #Log prediocally
    if iter%20==0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

In [ ]:
#Making the recommendations

#Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:, 0]

#Sort the predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f"Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}")

print("\nOriginal vs Predicted ratings:\n")

for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f"Original rating: {my_ratings[i]}, Predicted rating: {my_predictions[i]:0.2f}, for {movieList[i]}")

In [ ]:
#Using pandas for sorting

filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)